In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# מייבאים את המודול yfinance להורדת נתוני מניות
import yfinance as yf
# מייבאים את הפונקציה לחלק את הנתונים לאימונים ולבדיקות
from sklearn.model_selection import train_test_split
# מייבאים את מודל XGBoost לרגרסיה
from xgboost import XGBRegressor
# מייבאים את matplotlib לצורך יצירת גרפים
import matplotlib.pyplot as plt
# מייבאים את StandardScaler לנרמול הנתונים
from sklearn.preprocessing import StandardScaler
# מייבאים את pandas לצורך ניהול מסגרות נתונים
import pandas as pd
# מייבאים את numpy לצורך עבודה עם מערכים
import numpy as np
# מייבאים את matplotlib.dates לצורך עיצוב תאריכים בגרפים
import matplotlib.dates as mdates
# כל השגיאות ואחוז הדיוק במודל רגרסיה R^2
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error


# # רשימת הסימולים של המניות (תוקנה, הוסר מחרוזת ריקה)
tickers_df = pd.read_csv("/content/drive/Shareddrives/capstone project-stock market liquidity/Symbols.csv")
tickers = tickers_df['0'].tolist()

# # הורדת נתוני המניות בין התאריכים 01-01-2016 עד 01-09-2024
data = yf.download(tickers, start="2020-01-01", end="2024-12-23")['Adj Close']
data

[**********************61%****                   ]  312 of 510 completedERROR:yfinance:Could not get exchangeTimezoneName for ticker '???.TA' reason: 'chart'
[*********************100%***********************]  510 of 510 completed
ERROR:yfinance:
5 Failed downloads:
ERROR:yfinance:['HDHA.TA', '???.TA', 'PMCN.TA', 'UNCR.TA', 'UNVO.TA']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')


Ticker,???.TA,ABRA.TA,ACCL.TA,ACKR.TA,ACRO.TA,ADGR.TA,AFHL.TA,AFPR.TA,AFRE.TA,AICS-M.TA,...,XTLB.TA,YAAC.TA,YBOX.TA,YBRD.TA,YHNF.TA,ZMH.TA,ZNKL.TA,ZOOZ.TA,ZPRS.TA,ZUR.TA
Date,,,,,,,,,,,,,,,,,,,,,
2020-01-01,NaN,209.965805,50.000000,NaN,NaN,706.308472,13049.062500,14303.131836,8221.623047,11.000,...,0.046,41.237598,72.882301,11060.000000,15143.166016,1640.811279,1338.669067,NaN,NaN,1118.365479
2020-01-02,NaN,218.123596,54.200001,NaN,NaN,704.812805,12860.983398,14372.277344,8281.188477,11.550,...,0.048,40.750198,72.386497,10850.000000,15272.297852,1642.481567,1397.088989,NaN,NaN,1131.820557
2020-01-05,NaN,212.685104,51.900002,NaN,NaN,704.314087,12475.871094,14224.109375,8322.551758,11.250,...,0.046,42.115002,71.692398,10980.000000,15272.297852,1639.976685,1399.938843,NaN,NaN,1141.318726
2020-01-06,NaN,202.099396,33.299999,NaN,NaN,696.835510,12045.976562,14085.818359,8151.302246,11.050,...,0.045,41.432598,71.196602,11020.000000,15237.865234,1648.326538,1417.749512,NaN,NaN,1125.488770
2020-01-07,NaN,210.742798,51.400002,NaN,NaN,695.422913,12126.582031,14332.765625,8195.974609,10.750,...,0.045,41.432598,73.774696,10790.000000,15410.043945,1638.306519,1370.729004,NaN,NaN,1102.536011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-16,NaN,293.799988,134.300003,811.900024,5700.0,574.099976,16430.000000,17180.000000,28210.000000,3.279,...,0.057,35.299999,110.099998,971.599976,23200.000000,3817.000000,4234.000000,970.000000,1204.0,820.700012
2024-12-17,NaN,292.000000,134.100006,825.000000,5655.0,575.799988,16420.000000,17240.000000,28450.000000,3.589,...,0.056,36.500000,110.699997,999.900024,23350.000000,3816.000000,4226.000000,996.599976,1230.0,834.799988
2024-12-18,NaN,297.000000,132.300003,830.000000,5630.0,578.099976,17250.000000,17200.000000,27670.000000,3.611,...,0.052,35.900002,109.400002,973.000000,23480.000000,3816.000000,4222.000000,988.900024,1293.0,836.000000


In [3]:
nan_columns_2024_04_07 = data.loc["2024-04-07"][data.loc["2024-04-07"].isna()].index

for column in nan_columns_2024_04_07:
    data[column] = data[column].fillna(method="bfill")

# Getting rid off of stocks with less than 100 days
data = data.dropna(axis=1, thresh=100)
data.shape


<ipython-input-3-10f0c26eff13>:4: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data[column] = data[column].fillna(method="bfill")


(1223, 505)

In [4]:
# פונקציה לציור גרף המציג את המחירים בפועל מול המחירים החזויים
def plot_actual_vs_predicted(data, y_test, pred, model_name='Model'):
    plt.figure(figsize=(16, 8))  # הגדרת גודל הגרף

    # ודא שהאינדקס של הנתונים הוא בפורמט תאריך
    data.index = pd.to_datetime(data.index)

    # ציור המחירים בפועל (בכחול) והמחירים החזויים (באדום)
    plt.plot(data.index[-len(y_test):], y_test, label='Actual', color='blue', linewidth=2)
    plt.plot(data.index[-len(y_test):], pred, label='Predicted', color='red', linewidth=2)
    plt.fill_between(data.index[-len(y_test):], y_test, pred, color='gray', alpha=0.3)  # אזור הצללה בין תחזית למחירים בפועל

    plt.xlabel('Date', fontsize=14)  # תווית ציר X
    plt.ylabel('Price', fontsize=14)  # תווית ציר Y
    plt.title(f'{model_name}: Actual vs Predicted Prices Over Time', fontsize=16)  # כותרת הגרף
    plt.legend(fontsize=12)  # הוספת אגדה

    # עיצוב ציר ה-X לתצוגת תאריכים
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
    plt.gca().xaxis.set_major_locator(mdates.AutoDateLocator())

    plt.gcf().autofmt_xdate()  # סידור אוטומטי של התאריכים
    plt.tight_layout()  # התאמת הגרף לחלון התצוגה
    plt.grid(True, linestyle='--', alpha=0.7)  # הוספת רשת
    plt.show()

# מעדכנים את רשימת המניות לאחר הורדת מניות עם פחות מ 100 ימים בדאטא
tickers = data.columns.tolist()
# יצירת ממוצעים נעים (Rolling windows) עם טיפול בנתוני NaN
for window in [2, 5, 10, 20, 60]:  # עבור חלונות של 2, 5, 10, 20 ו-60 ימים
    for ticker in tickers:  # עבור כל סמל במניות
        # ממוצע נע
        data[f'{ticker}_rolling_{window}'] = data[ticker].rolling(window=window).mean().fillna(method='ffill').fillna(method='bfill')
        # סטיית תקן נעה
        data[f'{ticker}_rollingSTD_{window}'] = data[ticker].rolling(window=window).std().fillna(method='ffill').fillna(method='bfill')
        # חציון נע
        data[f'{ticker}_rollingMedian_{window}'] = data[ticker].rolling(window=window).median().fillna(method='ffill').fillna(method='bfill')

# # הוספת ממוצע נע אקספוננציאלי עבור החלונות הבאים
# for window in [20, 50, 100, 200]:
#     for ticker in tickers:
#         data[f'{ticker}_ema_{window}'] = data[ticker].ewm(span=window, adjust=False).mean()


# מילוי ערכי NaN עם הערך הקודם או הבא
data.fillna(method='bfill', inplace=True)
data.fillna(method='ffill', inplace=True)


Streaming output truncated to the last 5000 lines.
<ipython-input-4-d326ba6432c8>:37: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data[f'{ticker}_rollingMedian_{window}'] = data[ticker].rolling(window=window).median().fillna(method='ffill').fillna(method='bfill')
<ipython-input-4-d326ba6432c8>:37: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[f'{ticker}_rollingMedian_{window}'] = data[ticker].rolling(window=window).median().fillna(method='ffill').fillna(method='bfill')
<ipython-input-4-d326ba6432c8>:33: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data[f'{ticker}_rolling_{wi

In [5]:
data.shape

(1223, 8080)

In [6]:
import os
import json
import pandas as pd
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Define the output CSV file
output_csv_path = "model_XGBoost_metrics_and_predictions.csv"

# Get the list of stocks

###### Change here the number of stocks ######
stocks = data.columns[400:]

# Check if the output file exists
if os.path.exists(output_csv_path):
    # Load the existing CSV to find processed stocks
    existing_df = pd.read_csv(output_csv_path)
    processed_stocks = set(existing_df["Stock"].unique())
    stock_number_offset = len(existing_df)  # Start numbering from the existing number
else:
    # No file exists, start fresh
    processed_stocks = set()
    stock_number_offset = 0

# Loop through all stocks
num_skip = 0
for i, name_of_stock in enumerate(stocks, start=1):
    # Skip stocks that are already processed
    if name_of_stock in processed_stocks:
        print(f"Skipping already processed stock: {name_of_stock}")
        num_skip += 1
        continue

    print(f"Processing stock: {name_of_stock}")

    try:
        # Normalize independent variables (X), excluding the target stock
        scaler_X = StandardScaler()
        X_scaled = pd.DataFrame(scaler_X.fit_transform(data.drop(columns=[name_of_stock])),
                                columns=data.columns.drop(name_of_stock),
                                index=data.index)

        # Define the target variable
        y = data[name_of_stock]

        # Split data into training and testing sets (chronological order)
        X_train, X_test, y_train, y_test, train_dates, test_dates = train_test_split(
            X_scaled, y, X_scaled.index, test_size=0.2, random_state=42, shuffle=False)

        # Train the XGBoost model

        ###### Change here the model ######
        model = XGBRegressor(n_estimators=100, learning_rate=0.05, max_depth=6, random_state=42)
        model.fit(X_train, y_train)
        ######                       ######

        # Predict on test data
        y_pred = model.predict(X_test)

        # Calculate Metrics
        r2 = r2_score(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        mse = mean_squared_error(y_test, y_pred)

        # Prepare data for the last 30 days
        X_last_30 = X_scaled.iloc[-30:].copy()
        X_scaled = X_scaled.iloc[:-30]

        # Predict future values for the next 30 days
        y_future_pred = model.predict(X_last_30)

        # Prepare columns for future predictions (Day_1, Day_2, ..., Day_30)
        prediction_columns = {f"Day_{i+1}": y_future_pred[i] for i in range(len(y_future_pred))}

        # Create a single row of data for the stock
        metrics_data = {
            "Stock_Number": stock_number_offset + i - num_skip,  # Add stock number
            "Stock": name_of_stock,
            "R-squared": r2,
            "MAE": mae,
            "MSE": mse,
            **prediction_columns,  # Add the dynamic prediction columns
            "y_test": json.dumps(y_test.tolist()),  # Serialize y_test as a JSON string
            "y_pred": json.dumps(y_pred.tolist())   # Serialize y_pred as a JSON string
        }

        metrics_df = pd.DataFrame([metrics_data])

        # Append to CSV if it exists, otherwise create a new file
        if os.path.exists(output_csv_path):
            existing_df = pd.read_csv(output_csv_path)
            updated_df = pd.concat([existing_df, metrics_df], ignore_index=True)
            updated_df.to_csv(output_csv_path, index=False)
        else:
            metrics_df.to_csv(output_csv_path, index=False)

        print(f"Metrics and future predictions for {name_of_stock} saved to {output_csv_path}")

    except Exception as e:
        # Log the error and continue with the next stock
        print(f"An error occurred while processing stock {name_of_stock}: {e}")
        num_skip += 1


Skipping already processed stock: RSEL.TA
Skipping already processed stock: RTEN.TA
Skipping already processed stock: RTLS.TA
Skipping already processed stock: RTMD-L.TA
Skipping already processed stock: RTPT.TA
Skipping already processed stock: RVL.TA
Skipping already processed stock: RZR.TA
Skipping already processed stock: SAE.TA
Skipping already processed stock: SANO1.TA
Skipping already processed stock: SARN.TA
Skipping already processed stock: SBEN.TA
Skipping already processed stock: SCC.TA
Skipping already processed stock: SCDX.TA
Skipping already processed stock: SCOP.TA
Skipping already processed stock: SEMG.TA
Skipping already processed stock: SFRN.TA
Processing stock: SHAN.TA


/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1101: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1106: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1126: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


Metrics and future predictions for SHAN.TA saved to model_XGBoost_metrics_and_predictions.csv
Processing stock: SHGR.TA


/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1101: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1106: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1126: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


Metrics and future predictions for SHGR.TA saved to model_XGBoost_metrics_and_predictions.csv
Processing stock: SHMM.TA


...

Metrics and future predictions for AMAN.TA_rolling_2 saved to model_XGBoost_metrics_and_predictions.csv
Processing stock: AMAN.TA_rollingSTD_2


/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1101: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1106: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1126: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


Metrics and future predictions for AMAN.TA_rollingSTD_2 saved to model_XGBoost_metrics_and_predictions.csv
Processing stock: AMAN.TA_rollingMedian_2


/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1101: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1106: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1126: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


KeyboardInterrupt: 

In [ ]:
# import matplotlib.dates as mdates

# # פונקציה לציור תחזיות הימים הבאים עם פורמט תאריך מתאים
# def plot_future_predictions(future_dates, y_future_pred, model_name='Model'):
#     plt.figure(figsize=(8, 5))
#     plt.plot(future_dates, y_future_pred, label=f'{model_name} Predictions (Next 5 Days)', color='orange')

#     # הגדרת פורמט תאריך לציר ה-X
#     plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))

#     # הגדרת מרווחי התאריכים לציר ה-X (יום אחד בין כל תאריך)
#     plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=1))

#     # סיבוב התאריכים כדי שיתאימו לגודל הגרף
#     plt.gcf().autofmt_xdate()

#     plt.xlabel('Date')
#     plt.ylabel('Predicted BTC-USD % Change')
#     plt.title(f'{model_name} - Predicted BTC-USD for the Next 5 Days')
#     plt.legend()
#     plt.grid(False)
#     plt.show()

# # ציור הגרף
# plot_future_predictions(future_dates, y_future_pred, model_name='XGBoost With Memory')